In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('data/mimTitles.csv')
df.head()

,MIM Number,Preferred Title,symbol
0,100050,"AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN
1,100070,"AORTIC ANEURYSM, FAMILIAL ABDOMINAL, 1",AAA1
2,100100,PRUNE BELLY SYNDROME,PBS
3,100200,ABDUCENS PALSY,NaN
4,100300,ADAMS-OLIVER SYNDROME 1,AOS1


In [ ]:
import re

# Create a mapping of MIM Numbers to Preferred Titles
mim_to_title = dict(zip(df['MIM Number'], df['Preferred Title']))

# Function to get the final title for a MIM Number, handling circular references


def get_final_title(mim_number, visited=None):
    if visited is None:
        visited = set()

    if mim_number in visited:
        return f"[CIRCULAR REFERENCE: {mim_number}]"

    visited.add(mim_number)

    title = mim_to_title.get(mim_number)
    if isinstance(title, str) and 'MOVED TO' in title:
        match = re.search(r'MOVED TO (\d+)', title)
        if match:
            new_mim = int(match.group(1))
            return get_final_title(new_mim, visited)
    return title

# Function to replace "MOVED TO [MIM Number]" with the title of that MIM Number


def replace_moved_to(title):
    if isinstance(title, str) and 'MOVED TO' in title:
        match = re.search(r'MOVED TO (\d+)', title)
        if match:
            mim_number = int(match.group(1))
            final_title = get_final_title(mim_number)
            if final_title:
                return final_title
    return title


# Apply the function to update the 'Preferred Title' column
df['Preferred Title'] = df['Preferred Title'].apply(replace_moved_to)

# Display the first few rows to verify the changes
df.head()

,MIM Number,Preferred Title,symbol
0,100050,"AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN
1,100070,"AORTIC ANEURYSM, FAMILIAL ABDOMINAL, 1",AAA1
2,100100,PRUNE BELLY SYNDROME,PBS
3,100200,ABDUCENS PALSY,NaN
4,100300,ADAMS-OLIVER SYNDROME 1,AOS1


In [9]:
df.to_csv('data/mimTitles.csv')

In [8]:
# Count occurrences of "REMOVED FROM DATABASE"
removed_count = (df['Preferred Title'] == "REMOVED FROM DATABASE").sum()
print(f"Found {removed_count} entries with 'REMOVED FROM DATABASE'")

# Replace "REMOVED FROM DATABASE" with "TITLE NOT AVAILABLE AT THIS MOMENT"
df['Preferred Title'] = df['Preferred Title'].replace("REMOVED FROM DATABASE", "TITLE NOT AVAILABLE AT THIS MOMENT")

# Verify the changes
updated_count = (df['Preferred Title'] == "TITLE NOT AVAILABLE AT THIS MOMENT").sum()
print(f"Updated {updated_count} entries to 'TITLE NOT AVAILABLE AT THIS MOMENT'")

# Display a few examples if available
if updated_count > 0:
    print("\nExamples of updated entries:")
    print(df[df['Preferred Title'] == "TITLE NOT AVAILABLE AT THIS MOMENT"].head())

Found 118 entries with 'REMOVED FROM DATABASE'
Updated 118 entries to 'TITLE NOT AVAILABLE AT THIS MOMENT'

Examples of updated entries:
     MIM Number                     Preferred Title symbol
58       102570  TITLE NOT AVAILABLE AT THIS MOMENT    NaN
98       102920  TITLE NOT AVAILABLE AT THIS MOMENT    NaN
99       102930  TITLE NOT AVAILABLE AT THIS MOMENT    NaN
100      102940  TITLE NOT AVAILABLE AT THIS MOMENT    NaN
101      102970  TITLE NOT AVAILABLE AT THIS MOMENT    NaN


In [ ]:
df.to_csv('')